In [3]:
#### import os 
import sys 
sys.path.append("..")
import argparse

import math
import numpy as np
import torch
import torch.nn as nn

from SSB.utils import load_class_splits

from model import DINOHead, info_nce_logits, SupConLoss, DistillLoss, ContrastiveLearningViewGenerator, get_params_groups

/conor/SimGCD-ICCV-Challenge/SSB/SSB/utils.py:55: UserWarning: The dataset path is set to the default value. Consider setting it to a suitable path in your ~/.ssb/ssb_config.json file.
  warnings.warn("The dataset path is set to the default value. "


In [4]:
# backbone = torch.hub.load('facebookresearch/dino:main', 'dino_vitb16')
backbone = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14')

Using cache found in /root/.cache/torch/hub/facebookresearch_dinov2_main
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [5]:
backbone

DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0-11): 12 x NestedTensorBlock(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): MemEffAttention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (ls2): LayerScale()
      (drop_path2): Identity()
    )
  )
  (n

In [12]:
projector = DINOHead(in_dim=786, out_dim=200, nlayers=3)

model = nn.Sequential(backbone, projector)

In [17]:
model(torch.rand((128, 3, 224, 224)))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (128x768 and 786x2048)

In [15]:
backbone

DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0-11): 12 x NestedTensorBlock(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): MemEffAttention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (ls2): LayerScale()
      (drop_path2): Identity()
    )
  )
  (n

In [50]:

for name, m in backbone.named_parameters():
    if 'block' in name:
        block_num = int(name.split('.')[1])
        if block_num >= 11: #gardient from block 보다 노드가 많으면 학습해야 한단 거가? 
            m.requires_grad = True
            print(name)

blocks.11.norm1.weight
blocks.11.norm1.bias
blocks.11.attn.qkv.weight
blocks.11.attn.qkv.bias
blocks.11.attn.proj.weight
blocks.11.attn.proj.bias
blocks.11.ls1.gamma
blocks.11.norm2.weight
blocks.11.norm2.bias
blocks.11.mlp.fc1.weight
blocks.11.mlp.fc1.bias
blocks.11.mlp.fc2.weight
blocks.11.mlp.fc2.bias
blocks.11.ls2.gamma


In [3]:
backbone = torch.hub.load('facebookresearch/dino:main', 'dino_xcit_medium_24_p8')

Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main
Using cache found in /root/.cache/torch/hub/facebookresearch_xcit_main


In [4]:
backbone

XCiT(
  (patch_embed): ConvPatchEmbed(
    (proj): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): SyncBatchNorm(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): GELU(approximate='none')
      (2): Sequential(
        (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): SyncBatchNorm(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (3): GELU(approximate='none')
      (4): Sequential(
        (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): SyncBatchNorm(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0-23): 24 x XCABlock(
      (norm1): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
      (attn): XCA(
        (qkv):

In [5]:
cnt = 0
for name, m in backbone.named_parameters():
    if 'block' in name:
        block_num = int(name.split('.')[1])
        if block_num >= 23: #gardient from block 보다 노드가 많으면 학습해야 한단 거가? 
            cnt += 1
            print(name, cnt)


blocks.23.gamma1 1
blocks.23.gamma2 2
blocks.23.gamma3 3
blocks.23.norm1.weight 4
blocks.23.norm1.bias 5
blocks.23.attn.temperature 6
blocks.23.attn.qkv.weight 7
blocks.23.attn.qkv.bias 8
blocks.23.attn.proj.weight 9
blocks.23.attn.proj.bias 10
blocks.23.norm2.weight 11
blocks.23.norm2.bias 12
blocks.23.mlp.fc1.weight 13
blocks.23.mlp.fc1.bias 14
blocks.23.mlp.fc2.weight 15
blocks.23.mlp.fc2.bias 16
blocks.23.norm3.weight 17
blocks.23.norm3.bias 18
blocks.23.local_mp.conv1.weight 19
blocks.23.local_mp.conv1.bias 20
blocks.23.local_mp.bn.weight 21
blocks.23.local_mp.bn.bias 22
blocks.23.local_mp.conv2.weight 23
blocks.23.local_mp.conv2.bias 24


In [10]:
backbone(torch.rand((2, 3, 224,224)))

(tensor([[ 0.6354, -1.0563,  1.7515,  ...,  3.6439,  1.6347,  0.9784],
         [ 1.1464,  0.0533, -2.2170,  ...,  3.0749, -0.0769,  0.0210]],
        grad_fn=<SelectBackward0>),
 tensor([[ 0.6354, -1.0563,  1.7515,  ...,  3.6439,  1.6347,  0.9784],
         [ 1.1464,  0.0533, -2.2170,  ...,  3.0749, -0.0769,  0.0210]],
        grad_fn=<SelectBackward0>))

In [19]:
parser = argparse.ArgumentParser(description='cluster', formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument('--batch_size', default=128, type=int)
parser.add_argument('--num_workers', default=8, type=int)
parser.add_argument('--eval_funcs', nargs='+', help='Which eval functions to use', default=['v2', 'v2p'])

parser.add_argument('--warmup_model_dir', type=str, default=None)
parser.add_argument('--dataset_name', type=str, default='scars', help='options: cifar10, cifar100, imagenet_100, cub, scars, fgvc_aricraft, herbarium_19')
parser.add_argument('--prop_train_labels', type=float, default=0.5)
parser.add_argument('--use_ssb_splits', action='store_true', default=True)

parser.add_argument('--grad_from_block', type=int, default=11)
parser.add_argument('--lr', type=float, default=0.1)
parser.add_argument('--gamma', type=float, default=0.1)
parser.add_argument('--momentum', type=float, default=0.9)
parser.add_argument('--weight_decay', type=float, default=1e-4)
parser.add_argument('--epochs', default=200, type=int)
# parser.add_argument('--exp_root', type=str, default=exp_root)
parser.add_argument('--transform', type=str, default='imagenet')
parser.add_argument('--sup_weight', type=float, default=0.35)
parser.add_argument('--n_views', default=2, type=int)

parser.add_argument('--memax_weight', type=float, default=2)
parser.add_argument('--warmup_teacher_temp', default=0.07, type=float, help='Initial value for the teacher temperature.')
parser.add_argument('--teacher_temp', default=0.04, type=float, help='Final value (after linear warmup)of the teacher temperature.')
parser.add_argument('--warmup_teacher_temp_epochs', default=30, type=int, help='Number of warmup epochs for the teacher temperature.')

parser.add_argument('--fp16', action='store_true', default=False)
parser.add_argument('--print_freq', default=10, type=int)
parser.add_argument('--exp_name', default=None, type=str)
args = parser.parse_args(args=[])

### Backbone 비교 

In [20]:
backbone = torch.hub.load('facebookresearch/dino:main', 'dino_vitb16')

Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main


In [21]:
backbone

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0-11): 12 x Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (norm): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
  (head): Identity()
)

In [12]:
cnt = 0
for name, m in backbone.named_parameters():
    if 'block' in name:
        block_num = int(name.split('.')[1])
        if block_num >= 10: #gardient from block 보다 노드가 많으면 학습해야 한단 거가? 
            cnt += 1
            print(name, cnt)


blocks.10.norm1.weight 1
blocks.10.norm1.bias 2
blocks.10.attn.qkv.weight 3
blocks.10.attn.qkv.bias 4
blocks.10.attn.proj.weight 5
blocks.10.attn.proj.bias 6
blocks.10.norm2.weight 7
blocks.10.norm2.bias 8
blocks.10.mlp.fc1.weight 9
blocks.10.mlp.fc1.bias 10
blocks.10.mlp.fc2.weight 11
blocks.10.mlp.fc2.bias 12
blocks.11.norm1.weight 13
blocks.11.norm1.bias 14
blocks.11.attn.qkv.weight 15
blocks.11.attn.qkv.bias 16
blocks.11.attn.proj.weight 17
blocks.11.attn.proj.bias 18
blocks.11.norm2.weight 19
blocks.11.norm2.bias 20
blocks.11.mlp.fc1.weight 21
blocks.11.mlp.fc1.bias 22
blocks.11.mlp.fc2.weight 23
blocks.11.mlp.fc2.bias 24


#### Swin

In [5]:
# !cd .. && git clone https://github.com/microsoft/esvit.git
# !pip install einops
# !pip install protobuf==3.20
# !pip install -r ../esvit/requirements.txt
# in, models/__init__.py => swin_transformer 외에는 다 주석 처리함. 

In [25]:
# ckpt = torch.load("../checkpoint_best.pth", map_location='cpu')

In [8]:
ckpt = torch.load("../checkpoint_best.pth", map_location='cpu')

In [28]:
ckpt["teacher"]

OrderedDict([('patch_embed.proj.weight',
              tensor([[[[-7.8015e-03, -3.4860e-03, -1.9038e-03, -5.5182e-03],
                        [-1.1968e-03,  1.2282e-02,  1.1503e-02,  3.2299e-03],
                        [-2.3499e-03,  1.1441e-02,  1.5077e-02,  4.0177e-03],
                        [-1.3387e-02, -5.3152e-03, -2.2913e-03, -7.1819e-03]],
              
                       [[-1.0030e-02, -4.1026e-03, -3.6914e-03, -8.4597e-03],
                        [-7.9027e-04,  1.4701e-02,  1.3584e-02,  3.0855e-03],
                        [-2.1192e-03,  1.2731e-02,  1.5987e-02,  4.3862e-03],
                        [-1.5163e-02, -6.5713e-03, -3.0359e-03, -7.0077e-03]],
              
                       [[-7.3291e-03, -2.6344e-03, -2.3192e-03, -7.9739e-03],
                        [-4.4757e-04,  1.1529e-02,  1.0035e-02,  5.4016e-04],
                        [-8.1964e-04,  9.5360e-03,  1.2150e-02,  2.1228e-03],
                        [-1.0339e-02, -4.2765e-03, -2.0876e-03, -5.56

In [ ]:
# Please specify --arch, --cfg and --pretrained_weights

In [ ]:
eval_knn.py --data_path $DATA_PATH --dump_features $OUT_PATH/features/epoch0300 --pretrained_weights $CKPT_PATH --checkpoint_key teacher --batch_size_per_gpu 256 --arch swin_tiny --cfg experiments/imagenet/swin/swin_tiny_patch4_window7_224.yaml MODEL.NUM_CLASSES 0


In [2]:
import sys
sys.path.append("../")

from esvit.config import config
from esvit.config import update_config
from esvit.config import save_config

import esvit.utils as utils
from esvit.models import build_model


parser = argparse.ArgumentParser('Evaluation with weighted k-NN on ImageNet')

parser.add_argument('--cfg',
                    default="../esvit/experiments/imagenet/swin/swin_base_patch4_window14_224.yaml",
                    help='experiment configure file name',
                    type=str)

parser.add_argument('--arch', default='swin_base', type=str,
    choices=['cvt_tiny', 'swin_tiny','swin_small', 'swin_base', 'swin_large', 'swin', 'vil', 'vil_1281', 'vil_2262', 'deit_tiny', 'deit_small', 'vit_base'],
    help="""Name of architecture to train. For quick experiments with ViTs,
    we recommend using deit_tiny or deit_small.""")

parser.add_argument('--batch_size_per_gpu', default=256, type=int, help='Per-GPU batch-size')
parser.add_argument('--nb_knn', default=[10, 20, 100, 200], nargs='+', type=int,
    help='Number of NN to use. 20 is usually working the best.')

parser.add_argument('--temperature', default=0.07, type=float,
    help='Temperature used in the voting coefficient')

parser.add_argument('--pretrained_weights', default='', type=str, help="Path to pretrained weights to evaluate.")
parser.add_argument('--use_cuda', default=True, type=utils.bool_flag,
    help="Should we store the features on GPU? We recommend setting this to False if you encounter OOM")

parser.add_argument('--patch_size', default=16, type=int, help='Patch resolution of the model.')

parser.add_argument("--checkpoint_key", default="teacher", type=str,
    help='Key to use in the checkpoint (example: "teacher")')
parser.add_argument('--dump_features', default=None,
    help='Path where to save computed features, empty for no saving')
parser.add_argument('--load_features', default=None, help="""If the features have
    already been computed, where to find them.""")
parser.add_argument('--num_workers', default=10, type=int, help='Number of data loading workers per GPU.')
parser.add_argument("--dist_url", default="env://", type=str, help="""url used to set up
    distributed training; see https://pytorch.org/docs/stable/distributed.html""")
parser.add_argument("--local_rank", default=0, type=int, help="Please ignore and do not set this argument.")
parser.add_argument('--data_path', default='/path/to/imagenet/', type=str)

# Dataset
parser.add_argument('--zip_mode', type=utils.bool_flag, default=False, help="""Whether or not
    to use zip file.""")


parser.add_argument('opts',
                    help="Modify config options using the command-line",
                    default=None,
                    nargs=argparse.REMAINDER)    

args = parser.parse_args(args=[])

/conor/SimGCD-ICCV-Challenge/nbfiles/../esvit/models/swin_transformer.py:865: SyntaxWarning: "is" with a literal. Did you mean "=="?
  or pretrained_layers[0] is '*'
/conor/SimGCD-ICCV-Challenge/nbfiles/../esvit/models/swin_transformer.py:924: SyntaxWarning: "is" with a literal. Did you mean "=="?
  or (len(frozen_layers) > 0 and frozen_layers[0] is '*')
/conor/SimGCD-ICCV-Challenge/nbfiles/../esvit/models/swin_transformer.py:935: SyntaxWarning: "is" with a literal. Did you mean "=="?
  or (len(frozen_layers) > 0 and frozen_layers[0] is '*')


In [1]:
import sys
sys.path.append("../")


In [2]:
from esvit_model import load_model



In [3]:
model = load_model()

=> merge config from ../esvit/experiments/imagenet/swin/swin_base_patch4_window14_224.yaml


/usr/local/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Take key teacher in provided checkpoint dict
Pretrained weights found at /conor/SimGCD-ICCV-Challenge/checkpoint_best.pth and loaded with msg: _IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=['head_dense.mlp.0.weight', 'head_dense.mlp.0.bias', 'head_dense.mlp.2.weight', 'head_dense.mlp.2.bias', 'head_dense.mlp.4.weight', 'head_dense.mlp.4.bias', 'head_dense.last_layer.weight_g', 'head_dense.last_layer.weight_v', 'head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v'])


In [10]:
cnt = 0
for name, m in model.named_parameters():
    cnt += 1

    if 'head' in name:
#         block_num = int(name.split('.')[1])
#         if block_num >= 10: #gardient from block 보다 노드가 많으면 학습해야 한단 거가? 
#             cnt += 1
        print(name, cnt)

head.weight 328
head.bias 329


In [11]:
model

SwinTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
  )
  (pos_drop): Dropout(p=0, inplace=False)
  (layers): ModuleList(
    (0): BasicLayer(
      dim=128, input_resolution=(56, 56), depth=2
      (blocks): ModuleList(
        (0): SwinTransformerBlock(
          dim=128, input_resolution=(56, 56), num_heads=4, window_size=14, shift_size=0 mlp_ratio=4
          (norm1): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
          (attn): WindowAttention(
            dim=128, window_size=(14, 14), num_heads=4
            (qkv): Linear(in_features=128, out_features=384, bias=True)
            (attn_drop): Dropout(p=0, inplace=False)
            (proj): Linear(in_features=128, out_features=128, bias=True)
            (proj_drop): Dropout(p=0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (drop_path): Identity()
          (norm2): Laye

In [24]:
import sys
sys.path.append("../")

from esvit.config import config
from esvit.config import update_config
from esvit.config import save_config

import esvit.utils as utils
from esvit.models import build_model


parser = argparse.ArgumentParser('Evaluation with weighted k-NN on ImageNet')

parser.add_argument('--cfg',
                    default="../esvit/experiments/imagenet/swin/swin_base_patch4_window14_224.yaml",
                    help='experiment configure file name',
                    type=str)

parser.add_argument('--arch', default='swin_base', type=str,
    choices=['cvt_tiny', 'swin_tiny','swin_small', 'swin_base', 'swin_large', 'swin', 'vil', 'vil_1281', 'vil_2262', 'deit_tiny', 'deit_small', 'vit_base'],
    help="""Name of architecture to train. For quick experiments with ViTs,
    we recommend using deit_tiny or deit_small.""")

parser.add_argument('--pretrained_weights', default='', type=str, help="Path to pretrained weights to evaluate.")
# parser.add_argument('--pretrained_weights', default='../checkpoint_best.pth', type=str, help="Path to pretrained weights to evaluate.")
parser.add_argument('--use_cuda', default=True, type=utils.bool_flag)
# parser.add_argument('--patch_size', default=16, type=int, help='Patch resolution of the model.')
parser.add_argument("--local_rank", default=0, type=int, help="Please ignore and do not set this argument.")
parser.add_argument('opts',
                    help="Modify config options using the command-line",
                    default=None,
                    nargs=argparse.REMAINDER)    

args = parser.parse_args(args=[])

In [26]:
update_config(config, args)
model = build_model(config, is_teacher=True)

# model = model.load_state_dict(ckpt["teacher"], strict=False)
utils.load_pretrained_weights(model, 
                              "../checkpoint_best.pth", 
                              "teacher", 
                              "swin_base", 4)

=> merge config from ../esvit/experiments/imagenet/swin/swin_base_patch4_window14_224.yaml
Take key teacher in provided checkpoint dict
Pretrained weights found at ../checkpoint_best.pth and loaded with msg: _IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=['head_dense.mlp.0.weight', 'head_dense.mlp.0.bias', 'head_dense.mlp.2.weight', 'head_dense.mlp.2.bias', 'head_dense.mlp.4.weight', 'head_dense.mlp.4.bias', 'head_dense.last_layer.weight_g', 'head_dense.last_layer.weight_v', 'head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v'])


In [21]:
cnt = 0
for name, m in model.named_parameters():
    cnt += 1
    print(name)
    
print(cnt)
#     if 'block' in name:
#         block_num = int(name.split('.')[1])
#         if block_num >= 10: #gardient from block 보다 노드가 많으면 학습해야 한단 거가? 
#             cnt += 1
#             print(name, cnt)


patch_embed.proj.weight
patch_embed.proj.bias
patch_embed.norm.weight
patch_embed.norm.bias
layers.0.blocks.0.norm1.weight
layers.0.blocks.0.norm1.bias
layers.0.blocks.0.attn.relative_position_bias_table
layers.0.blocks.0.attn.qkv.weight
layers.0.blocks.0.attn.qkv.bias
layers.0.blocks.0.attn.proj.weight
layers.0.blocks.0.attn.proj.bias
layers.0.blocks.0.norm2.weight
layers.0.blocks.0.norm2.bias
layers.0.blocks.0.mlp.fc1.weight
layers.0.blocks.0.mlp.fc1.bias
layers.0.blocks.0.mlp.fc2.weight
layers.0.blocks.0.mlp.fc2.bias
layers.0.blocks.1.norm1.weight
layers.0.blocks.1.norm1.bias
layers.0.blocks.1.attn.relative_position_bias_table
layers.0.blocks.1.attn.qkv.weight
layers.0.blocks.1.attn.qkv.bias
layers.0.blocks.1.attn.proj.weight
layers.0.blocks.1.attn.proj.bias
layers.0.blocks.1.norm2.weight
layers.0.blocks.1.norm2.bias
layers.0.blocks.1.mlp.fc1.weight
layers.0.blocks.1.mlp.fc1.bias
layers.0.blocks.1.mlp.fc2.weight
layers.0.blocks.1.mlp.fc2.bias
layers.0.downsample.reduction.weight
lay

In [20]:
cnt = 0
for name, m in backbone.named_parameters():
    cnt += 1
    print(name)
    
print(cnt)

cls_token
pos_embed
patch_embed.proj.weight
patch_embed.proj.bias
blocks.0.norm1.weight
blocks.0.norm1.bias
blocks.0.attn.qkv.weight
blocks.0.attn.qkv.bias
blocks.0.attn.proj.weight
blocks.0.attn.proj.bias
blocks.0.norm2.weight
blocks.0.norm2.bias
blocks.0.mlp.fc1.weight
blocks.0.mlp.fc1.bias
blocks.0.mlp.fc2.weight
blocks.0.mlp.fc2.bias
blocks.1.norm1.weight
blocks.1.norm1.bias
blocks.1.attn.qkv.weight
blocks.1.attn.qkv.bias
blocks.1.attn.proj.weight
blocks.1.attn.proj.bias
blocks.1.norm2.weight
blocks.1.norm2.bias
blocks.1.mlp.fc1.weight
blocks.1.mlp.fc1.bias
blocks.1.mlp.fc2.weight
blocks.1.mlp.fc2.bias
blocks.2.norm1.weight
blocks.2.norm1.bias
blocks.2.attn.qkv.weight
blocks.2.attn.qkv.bias
blocks.2.attn.proj.weight
blocks.2.attn.proj.bias
blocks.2.norm2.weight
blocks.2.norm2.bias
blocks.2.mlp.fc1.weight
blocks.2.mlp.fc1.bias
blocks.2.mlp.fc2.weight
blocks.2.mlp.fc2.bias
blocks.3.norm1.weight
blocks.3.norm1.bias
blocks.3.attn.qkv.weight
blocks.3.attn.qkv.bias
blocks.3.attn.proj.wei

In [ ]:
# NOTE: Hardcoded image size as we do not finetune the entire ViT model
args.image_size = 224
args.feat_dim = 768
args.num_mlp_layers = 3
# args.mlp_out_dim = args.num_labeled_classes + args.num_unlabeled_classes
# import ipdb; ipdb.set_trace()

# cub, scars, fgvc_aricraft
class_splits = load_class_splits("cub")
args.train_classes = class_splits['known_classes']
args.mlp_out_dim = len(class_splits['known_classes']) \
                 + len(class_splits['unknown_classes']['Easy']) \
                 + len(class_splits['unknown_classes']['Medium']) \
                 + len(class_splits['unknown_classes']['Hard'])

# ----------------------
# HOW MUCH OF BASE MODEL TO FINETUNE

# ----------------------
for m in backbone.parameters():
    m.requires_grad = False

# Only finetune layers from block 'args.grad_from_block' onwards
for name, m in backbone.named_parameters():
    if 'block' in name:
        block_num = int(name.split('.')[1])
        if block_num >= args.grad_from_block: #gardient from block 보다 노드가 많으면 학습해야 한단 거가? 
            m.requires_grad = True

In [9]:
projector = DINOHead(in_dim=args.feat_dim, out_dim=args.mlp_out_dim, nlayers=args.num_mlp_layers)
model = nn.Sequential(backbone, projector)

In [10]:
model

Sequential(
  (0): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0-11): 12 x Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (norm): LayerNorm((7